In [63]:
import numpy as np
from tenpy.networks.mps import MPS
from tenpy.networks.site import SpinHalfFermionSite
from tenpy.models.lattice import Square
from tenpy.algorithms import dmrg
from tenpy.models.model import CouplingModel, CouplingMPOModel

import pickle
import matplotlib.pyplot as plt

In [62]:

# Define the lattice parameters
Lx, Ly = 4, 4  # dimensions of the lattice

# Create the SpinHalfFermionSite
site = SpinHalfFermionSite() 

# Create the square lattice with the SpinHalfFermionSite
lattice = Square(Lx, Ly, site, bc=['open', 'periodic'])


lattice.boundary_conditions

['open', 'periodic']

In [64]:


class CustomFermiHubbardModel(CouplingMPOModel):
    def __init__(self, params):
        self.tx = params.pop('tx', 1.0)
        self.ty = params.pop('ty', 1.0)
        self.U = params.pop('U', 1.0)
        self.mu = params.pop('mu', 0.0)
        super().__init__(params)

    def init_terms(self, params):
        for u1, u2, dx in self.lat.pairs['nearest_neighbors']:
            if dx[0] != 0:  # Hopping in the x-direction
                self.add_coupling(self.tx, u1, 'Cdu', u2, 'Cu', dx, plus_hc=True)
                self.add_coupling(self.tx, u1, 'Cdd', u2, 'Cd', dx, plus_hc=True)
            if dx[1] != 0:  # Hopping in the y-direction
                self.add_coupling(self.ty, u1, 'Cdu', u2, 'Cu', dx, plus_hc=True)
                self.add_coupling(self.ty, u1, 'Cdd', u2, 'Cd', dx, plus_hc=True)
        for u in range(len(self.lat.unit_cell)):
            self.add_onsite(self.U, u, 'NuNd')
            self.add_onsite(self.mu, u, 'Ntot')

# Define the model parameters
model_params = {
    'lattice': lattice,
    'tx': -1.0,  # Hopping amplitude in the x-direction
    'ty': -0.5,  # Hopping amplitude in the y-direction
    'U': 4.0,    # On-site interaction strength
    'mu': 0.0,   # Chemical potential
}

# Create the custom Fermi-Hubbard model
model = CustomFermiHubbardModel(model_params)

# Print the model to verify
print(model)


In [65]:


def run_dmrg_2d(Lx, Ly, U, tx, ty, mu, chi_max, sweeps, charge=0):
    # Define the lattice and SpinHalfFermionSite without any conservation law for simplicity
    site = SpinHalfFermionSite()
    lattice = Square(Lx, Ly, site, bc=['open', 'periodic'])

    # Set up the model parameters
    model_params = {
        'lattice': lattice,
        'tx': tx,  # Hopping amplitude in the x-direction
        'ty': ty,  # Hopping amplitude in the y-direction
        'U': U,
        'mu': mu
    }

    model = CustomFermiHubbardModel(model_params)

    # Initialize with an alternating pattern of filled and empty sites appropriate for 2D
    initial_state = []
    for x in range(Lx):
        for y in range(Ly):
            if (x + y) % 2 == 0:
                initial_state.append('up')
            else:
                initial_state.append('down')

    # Depending on charge modify the last site
    if charge == 1:
        initial_state[-1] = 'full'
    elif charge == -1:
        initial_state[-1] = 'empty'

    psi = MPS.from_product_state(model.lat.mps_sites(), initial_state, bc='finite')

    dmrg_params = {
        'mixer': True,
        'trunc_params': {
            'chi_max': chi_max,
            'svd_min': 1.e-8
        },
        'max_sweeps': sweeps,
    }

    eng = dmrg.TwoSiteDMRGEngine(psi, model, dmrg_params)
    E0, psi = eng.run()
    return E0

# Example usage
Lx, Ly = 2, 4
U = 4.0
tx = -1.0
ty = -1.0*np.exp(1j*np.pi/2)
mu = 0.0
chi_max = 1000
sweeps = 40
charge = 0

ground_state_energy = run_dmrg_2d(Lx, Ly, U, tx, ty, mu, chi_max, sweeps, charge)/(Ly*Lx)
print("Ground state energy:", ground_state_energy)


Ground state energy: -0.7442795851321635
